In [ ]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from typing import List

In [ ]:
URL_DATA = ["https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Arrivi-negli-agriturismi-in-Italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Arrivi-negli-esercizi-alberghieri-in-Italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Arrivi-nei-campeggi-e-villaggi-turistici-in-italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Presenze-negli-agriturismi-in-Italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Presenze-negli-esercizi-alberghieri-in-Italia-per-regione.csv",
            "https://raw.githubusercontent.com/FabioGagliardiIts/datasets/main/dati_turismo/Presenze-nei-campeggi-e-villaggi-turistici-in-italia-per-regione.csv"]

df_arrivi_agriturismi = pd.read_csv(URL_DATA[0]) # Prendiamo i dati degli arrivi e delle presenze
df_presenze_agriturismi = pd.read_csv(URL_DATA[3])
df_agriturismi = pd.merge(df_arrivi_agriturismi, df_presenze_agriturismi, on=['Regione', 'Anno'], how='inner') # Creiamo un unico dataframe per categoria

df_arrivi_alberghi = pd.read_csv(URL_DATA[1])
df_presenze_alberghi = pd.read_csv(URL_DATA[4])
df_alberghi = pd.merge(df_arrivi_agriturismi, df_presenze_agriturismi, on=['Regione', 'Anno'], how='inner')

df_arrivi_campeggi = pd.read_csv(URL_DATA[2])
df_presenze_campeggi = pd.read_csv(URL_DATA[5])
df_campeggi = pd.merge(df_arrivi_agriturismi, df_presenze_agriturismi, on=['Regione', 'Anno'], how='inner')

display(df_agriturismi)
display(df_alberghi)
display(df_campeggi)

KeyError: 'Regione'

In [ ]:
class CleanDataFrame:
  def __init__(self, data: DataFrame):
    self.data = data.dropna()

  def get_df(self):
    return self.data

  def get_colonne(self, colonne: List[str]):
    return self.data[colonne]

  def get_regione_by_nome(self, regioneRic: str):
    return self.data[self.data.Regione==regioneRic]
